# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

In [1]:
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/parsed.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/quakes.db

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/tsunamis.csv

!mkdir data

!mv *.csv data

!mv *.db data

--2024-09-14 06:23:27--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3515656 (3.4M) [text/plain]
Saving to: ‘earthquakes.csv’

earthquakes.csv     100%[===================>]   3.35M  --.-KB/s    in 0.08s   

2024-09-14 06:23:28 (42.9 MB/s) - ‘earthquakes.csv’ saved [3515656/3515656]

--2024-09-14 06:23:28--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.1

### Get Data from API

In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1726295810000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-14&endtime=2024-09-13',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 9781}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.08,
  'place': '17 km NW of Morongo Valley, CA',
  'time': 1726185557670,
  'updated': 1726247514980,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci40732231',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci40732231&format=geojson',
  'felt': 2,
  'cdi': 2.2,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 18,
  'net': 'ci',
  'code': '40732231',
  'ids': ',ci40732231,',
  'sources': ',ci,',
  'types': ',dyfi,focal-mechanism,nearby-cities,origin,phase-data,scitech-link,',
  'nst': 38,
  'dmin': 0.121,
  'rms': 0.13,
  'gap': 95,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.1 - 17 km NW of Morongo Valley, CA'},
 'geometry': {'type': 'Point', 'coordinates': [-116.7156667, 34.146, 8.69]},
 'id': 'ci40732231'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.08,"17 km NW of Morongo Valley, CA",1726185557670,1726247514980,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,2.2,NaN,...,",ci40732231,",",ci,",",dyfi,focal-mechanism,nearby-cities,origin,pha...",38.0,0.12100,0.13,95.0,ml,earthquake,"M 1.1 - 17 km NW of Morongo Valley, CA"
1,1.70,"29 km NNW of Susitna, Alaska",1726185225880,1726185297697,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak024brrl9to,",",ak,",",origin,phase-data,",NaN,NaN,0.30,NaN,ml,earthquake,"M 1.7 - 29 km NNW of Susitna, Alaska"
2,4.70,"44 km ESE of Calama, Chile",1726185085685,1726204081040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000ndee,",",us,",",origin,phase-data,",69.0,0.99800,0.49,48.0,mb,earthquake,"M 4.7 - 44 km ESE of Calama, Chile"
3,0.98,"20 km ESE of Lopez Point, CA",1726184960540,1726216116394,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75061146,",",nc,",",nearby-cities,origin,phase-data,",19.0,0.07461,0.07,127.0,md,earthquake,"M 1.0 - 20 km ESE of Lopez Point, CA"
4,5.20,"Izu Islands, Japan region",1726184894614,1726202652040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",usauto7000nded,us7000nded,",",usauto,us,",",internal-moment-tensor,origin,phase-data,",109.0,2.50900,0.70,27.0,mb,earthquake,"M 5.2 - Izu Islands, Japan region"


### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>